In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import os

features = ['f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16']


train_files = ['tube1.csv', 'tube2.csv', 'tube3.csv']
train_dfs = []

for file in train_files:
    df = pd.read_csv(file)
    df['label'] = (df['f7'] * df['f8'] > 0.1).astype(int)
    df = df.dropna(subset=features)
    df = df[df['label'] == 0] 
    train_dfs.append(df)

train_df = pd.concat(train_dfs, ignore_index=True)
X_train_raw = train_df[features].values


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)


class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 4)
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, input_dim)
        )
    def forward(self, x):
        return self.decoder(self.encoder(x))

model = Autoencoder(input_dim=X_train_tensor.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 50
for epoch in range(epochs):
    output = model(X_train_tensor)
    loss = criterion(output, X_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")
        
test_files = ['tube4.csv', 'tube5.csv']

for file in test_files:
    print(f"\n🔍 Testing {file}")
    df = pd.read_csv(file)
    df['label'] = (df['f7'] * df['f8'] > 0.1).astype(int)
    df = df.dropna(subset=features)
    
    X_test_raw = df[features].values
    y_true = df['label'].values
    X_test_scaled = scaler.transform(X_test_raw)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)


    with torch.no_grad():
        reconstructed = model(X_test_tensor)
        mse = torch.mean((X_test_tensor - reconstructed) ** 2, dim=1).numpy()


    threshold = np.percentile(mse, 95)
    y_pred = (mse > threshold).astype(int)

    print(classification_report(y_true, y_pred, digits=4))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


Epoch 10/50, Loss: 1.044730
Epoch 20/50, Loss: 1.018789
Epoch 30/50, Loss: 0.992624
Epoch 40/50, Loss: 0.962956
Epoch 50/50, Loss: 0.927732

🔍 Testing tube4.csv
              precision    recall  f1-score   support

           0     0.5972    1.0000    0.7478     33428
           1     1.0000    0.1156    0.2072     25492

    accuracy                         0.6173     58920
   macro avg     0.7986    0.5578    0.4775     58920
weighted avg     0.7715    0.6173    0.5139     58920

Confusion Matrix:
 [[33428     0]
 [22546  2946]]

🔍 Testing tube5.csv
              precision    recall  f1-score   support

           0     0.5764    1.0000    0.7313    183309
           1     1.0000    0.1105    0.1991    151433

    accuracy                         0.5976    334742
   macro avg     0.7882    0.5553    0.4652    334742
weighted avg     0.7681    0.5976    0.4905    334742

Confusion Matrix:
 [[183309      0]
 [134695  16738]]


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import os

features = ['f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16']

train_files = ['tube1.csv', 'tube2.csv', 'tube3.csv']
train_dfs = []

for file in train_files:
    df = pd.read_csv(file)
    f7_rounded = df['f7'].round(6)
    df['label'] = (~f7_rounded.isin([0.857143, 0.714286])).astype(int)
    df = df.dropna(subset=features)
    df = df[df['label'] == 0]  
    train_dfs.append(df)

train_df = pd.concat(train_dfs, ignore_index=True)
X_train_raw = train_df[features].values

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)

# Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 4)
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, input_dim)
        )
    def forward(self, x):
        return self.decoder(self.encoder(x))

model = Autoencoder(input_dim=X_train_tensor.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 3. 训练模型
epochs = 40
for epoch in range(epochs):
    output = model(X_train_tensor)
    loss = criterion(output, X_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

test_files = ['tube4.csv', 'tube5.csv']

for file in test_files:
    print(f"\n🔍 Testing {file}")
    df = pd.read_csv(file)
    f7_rounded = df['f7'].round(6)
    df['label'] = (~f7_rounded.isin([0.857143, 0.714286])).astype(int)
    df = df.dropna(subset=features)
    
    X_test_raw = df[features].values
    y_true = df['label'].values
    X_test_scaled = scaler.transform(X_test_raw)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

    # reconstruct error
    with torch.no_grad():
        reconstructed = model(X_test_tensor)
        mse = torch.mean((X_test_tensor - reconstructed) ** 2, dim=1).numpy()

    # threshold 95 percentile
    threshold = np.percentile(mse, 98)
    y_pred = (mse > threshold).astype(int)

    print(classification_report(y_true, y_pred, digits=4))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


Epoch 10/40, Loss: 1.063972
Epoch 20/40, Loss: 1.030316
Epoch 30/40, Loss: 1.002792
Epoch 40/40, Loss: 0.977843

🔍 Testing tube4.csv
              precision    recall  f1-score   support

           0     0.9302    0.9871    0.9578     54415
           1     0.4029    0.1054    0.1671      4505

    accuracy                         0.9197     58920
   macro avg     0.6665    0.5463    0.5625     58920
weighted avg     0.8899    0.9197    0.8973     58920

Confusion Matrix:
 [[53711   704]
 [ 4030   475]]

🔍 Testing tube5.csv
              precision    recall  f1-score   support

           0     0.9196    0.9885    0.9528    305169
           1     0.4780    0.1082    0.1765     29573

    accuracy                         0.9108    334742
   macro avg     0.6988    0.5484    0.5646    334742
weighted avg     0.8806    0.9108    0.8842    334742

Confusion Matrix:
 [[301674   3495]
 [ 26373   3200]]
